# 기본설정

In [3]:
# 경고를 무시합니다.
import warnings
warnings.filterwarnings("ignore")

# 환경변수 저장 (.env) 확인
from dotenv import load_dotenv
import os
load_dotenv()

# 블로그 기본사항을 입력합니다.
blogname = 'pasta_blog' # 명칭은 영어로 지정해주어야 합니다.

""" 테마는 실제 블로그의 테마이기 보다는 수집할 키워드들의 테마입니다.
파스타는 예시사항입니다.
1. 저는 블로그에 파스타와 관련된 사항들을 작성합니다.
2. 키워드 '파스타'로부터 모든 키워드들을 끌어모을 작정입니다.
3. 키워드의 블로그 게시 적정성 판단기준이 '파스타'가 됩니다.

파스타가 아니라 '여행', '영화' 또는 '여행과 영화' 와 같이 2개 이상의 주제를 가져도 좋습니다.
"""

theme = '파스타'

In [4]:

# 정의한 클래스들을 호출합니다.
from module.Crawler import Crawler
from module.AIAgent import AIAgent
from module.File_manager import File_manager
from module.Uploader import Uploader

# 클래스의 인스턴스들을 설정합니다.
# 크롤러 클래스 호출
crawler = Crawler(verbose=True)
# 파일매니저 클래스 호출
file_manager = File_manager(
    blogname=blogname, 
    verbose=True) # 현재위치에서 시작을 가정합니다.
# 인공지능 클래스 호출 - 포스팅용
posting_ai = AIAgent(
        blogname = blogname, 
        index = 'my_contents',             
        verbose=True)
# 인공지능 클래스 호출 - 키워드관리용
keyword_ai = AIAgent(
        blogname = blogname, 
        index = 'my_keywords',             
        verbose=True)
# 업로더 클래스 호출
uploader = Uploader(
    blogname=blogname, # 아이디와 비밀번호를 지정하지 않으면 환경변수에서 가져옵니다.        
    verbose=True)    

[WinError 183] 파일이 이미 있으므로 만들 수 없습니다: 'c:\\hellodear\\tistory_ebook\\pasta_blog'
[WinError 183] 파일이 이미 있으므로 만들 수 없습니다: 'c:\\hellodear\\tistory_ebook\\pasta_blog\\업로드할것'
[WinError 183] 파일이 이미 있으므로 만들 수 없습니다: 'c:\\hellodear\\tistory_ebook\\pasta_blog\\업로드완료'
[WinError 183] 파일이 이미 있으므로 만들 수 없습니다: 'c:\\hellodear\\tistory_ebook\\pasta_blog\\창고'
✅ 기존 벡터스토어 로드되었습니다.
✅ 기존 벡터스토어 로드되었습니다.


## 2.2. 로컬파일로 저장한 키워드 다시 불러오기

벡터스토어를 활용하기 위해서는 충분한 양의 키워드가 누적되어야 합니다.\
미리 준비해 둔 키워드들을 다운로드 받아서 활용하겠습니다.

In [5]:
collected_keywords_info = file_manager.get_keywords(what_keywords='collected_keywords') # 수집한 전체키워드 정보
collected_keywords = collected_keywords_info.index.to_list()
screened_keywords_info = file_manager.get_keywords(what_keywords='screened_keywords') # 수집한 전체키워드 정보
screened_keywords = screened_keywords_info.index.to_list()

print('총 키워드의 갯수 : ', len(collected_keywords))
print('선별 키워드의 갯수 : ', len(screened_keywords))
display(collected_keywords_info.head())

Data loaded from CSV:
Data loaded from CSV:
총 키워드의 갯수 :  364
선별 키워드의 갯수 :  74


,source,num_ads,all_links,tistory_rank_at_google,top_tistory_at_google
드라마 파스타,"['daum_open', 'daum_open', 'daum_open', 'daum_...",4,['https://namu.wiki/w/%ED%8C%8C%EC%8A%A4%ED%83...,노출없음,노출없음
파스타 종류,"['daum_open', 'daum_open', 'daum_suggest', 'da...",21,['https://namu.wiki/w/%ED%8C%8C%EC%8A%A4%ED%83...,20,https://huni1013.tistory.com/202
파스타 만들기,"['daum_open', 'daum_open', 'google_open', 'dau...",21,"['https://www.10000recipe.com/recipe/6865946',...",노출없음,노출없음
파스타 맛집,"['daum_open', 'daum_open', 'daum_suggest', 'da...",15,['https://m.post.naver.com/viewer/postView.nav...,노출없음,노출없음
토마토 파스타,"['daum_open', 'daum_open', 'google_open', 'goo...",24,"['https://www.10000recipe.com/recipe/6865946',...",13,https://pangsoon.tistory.com/214


# 3. 글작성하기 (AIAgent 클래스) 
- 3.1 키워드 적정성 판단
- 3.2 벡터스토어
    - 3.2.1. 벡터스토어 생성
    - 3.2.2. 벡터스토어에 키워드 저장
    - 3.2.3. 벡터스토 유사성 검색을 통한 유사키워드 선정
- 3.3 글작성하기
    - 3.3.1. 소제목 
    - 3.3.2. 프롤로그
    - 3.3.3. 제목
    - 3.3.4. 자료수집
    - 3.3.5. 본문 
    - 3.3.6. 이미지수집
- 3.4 포맷팅하기 (HTML 형식 만들기)
- 3.5 저장하기


## 3.1. 키워드 적정성 판단

In [6]:
suitable_keywords = keyword_ai.suitability_checker([theme] * len(screened_keywords), screened_keywords)
suitable_keywords_info = screened_keywords_info.loc[suitable_keywords]

file_manager.saveGeneratedDict2CSV(
    subject = blogname, 
    language='ko', 
    contents='suitable_keywords', 
    folder_category='storage', 
    data=suitable_keywords_info.to_dict())

c:\hellodear\tistory_ebook\venv\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


❌ 피스타치오 콜드브루 적정성 : no
❌ 피스타치오 크림라떼 적정성 : no
✅ 파스타 소스 소비기한 적정성 : yes
❌ 피스타치오 크림 콜드브루 적정성 : no
✅ 스파게티와 파스타의 차이점 적정성 : yes
✅ 초간단 스파게티 만들기 적정성 : yes
✅ 파스타와 스파게티의 차이점 적정성 : yes
✅ 파스타 종류 레시피 적정성 : yes
✅ 다이소 파스타냄비 적정성 : yes
✅ 이탈리아 파스타 역사 면 종류 적정성 : yes
❌ 피스타치오 아보카도 초콜릿 프라푸치노 적정성 : no
✅ 크림 파스타 종류 적정성 : yes
✅ 파스타 샐러드 만들기 적정성 : yes
✅ 파스타 소스 없을때 적정성 : yes
✅ 롤링파스타 까르보나라 적정성 : yes
❌ 피스타치오 부작용 적정성 : no
✅ 파스타 삶기 종류 적정성 : yes
✅ 강릉 파스타 맛집 적정성 : yes
✅ 파스타 스파게티 차이 적정성 : yes
✅ 토마토 파스타 스파게티 차이 적정성 : yes
❌ 피스타치오 하루 권장량 적정성 : no
✅ 일산 파스타 맛집 적정성 : yes
✅ 대전 파스타 맛집 적정성 : yes
✅ 파스타 종류별 삶기 적정성 : yes
❌ 피스타치오 하루 섭취량 적정성 : no
✅ 롤링파스타 토마토파스타 적정성 : yes
✅ 스파게티 메뉴 적정성 : yes
✅ 파스타 면 미리 삶기 적정성 : yes
✅ 파스타 면 빨리 삶기 적정성 : yes
✅ 스파게티 면 삶기 소금 적정성 : yes
✅ 통밀 파스타 삶는 시간 적정성 : yes
✅ 삶은 파스타면 칼로리 적정성 : yes
✅ 파스타면 종류별 삶는 시간 적정성 : yes
✅ 전주 파스타 맛집 적정성 : yes
✅ 전주 프리모파스타 메뉴 적정성 : yes
✅ 파스타 맛집 홍대 적정성 : yes
✅ 스파게티 의 특징 적정성 : yes
✅ 우유로 크림파스타 만들기 적정성 : yes
✅ 파스타 종류별 요리법 적정성 : yes
✅ 생토마토 파스타 적정성 : yes
❌ 쫄면 성분 적정성 : no
✅ 드라마 파스타 결말 적정성 : yes
✅

In [7]:
print('총 키워드의 갯수 : ', len(collected_keywords))
print('선별 키워드의 갯수 : ', len(screened_keywords))
print('적정 키워드의 갯수 : ', len(suitable_keywords))

총 키워드의 갯수 :  364
선별 키워드의 갯수 :  74
적정 키워드의 갯수 :  64


## 3.2. 벡터스토어

### 3.2.1 벡터스토어 생성

### 3.2.2. 벡터스토어에 키워드 저장

In [8]:
# 키워드 벡터스토어 저장 (keyword_ai)
keyword_ai.vectorstore_save_texts(suitable_keywords)

✅ 벡토어스토어 내 새 문서 추가 완료


,data,id
0,파스타 소스 소비기한,1e683d99-c4dd-4c58-b9a4-ff8767ce7895
1,스파게티와 파스타의 차이점,e280edbc-e930-4bd8-9fe6-f16dcb650242
2,초간단 스파게티 만들기,a3511074-cc57-41cf-9b89-dee82f2533c3
3,파스타와 스파게티의 차이점,9eccd119-bed5-484d-8be5-7346fd7d2ddb
4,파스타 종류 레시피,c83a7b14-a2a8-4bda-a2b4-82d3324ae3af
...,...,...
123,파스타면 성분표,7f3c19b3-3d5c-4d9c-bb61-1ca60a66732a
124,폰타나 파스타 소스 레시피,481302f7-0ed3-4c40-b386-3e5877c88a94
125,크림파스타 소스 활용,40b9fc3f-5897-4150-80da-5681a315dd4b
126,크림파스타 칼로리,67756c4b-0d0f-429a-9838-8e69d5819381


### 3.2.3. 벡터스토 유사성 검색을 통한 유사키워드 선정

In [9]:
# 글을 쓸 첫번째 키워드를 가져옵니다.
my_keyword = suitable_keywords[0]
print(f'🟢 이번에 사용할 키워드는 "{my_keyword}"입니다.')

🟢 이번에 사용할 키워드는 "파스타 소스 소비기한"입니다.


In [10]:
# 관련 키워드 선별 및 벡터스토어 삭제 확인 extract
my_keywords = keyword_ai.vectorstore_similarity_search(my_keyword, score_threshold=0.55)
print(f"🟢 {my_keyword} 연관 키워드이자 사용할 키워드 : {my_keywords}")


🟢 파스타 소스 소비기한 연관 키워드이자 사용할 키워드 : ['파스타 소스 소비기한', '파스타 소스 소비기한', '파스타 소스 없을때', '파스타 소스 없을때']


## 3.3 글작성하기

### 3.3.1. 소제목 

In [11]:
# 소제목 3개 생성
topics = posting_ai.create_subject('파스타', my_keywords, num_topic=3, save=True)
print("🟢 topic",topics)

🟢 topic ['파스타 소스 보관 방법', '파스타 소스 종류', '파스타 소스 대체품']


### 3.3.2. 프롤로그

In [12]:
# 프롤로그 생성
prologue = posting_ai.create_prologue(posting_ai.results['topics'], posting_ai.results['keywords'], save=True)
print("🟢 prologue",prologue)

🟢 prologue 파스타 소스는 다양한 요리에 활용되는 중요한 재료 중 하나입니다. 이 블로그 포스트에서는 **파스타 소스 소비기한**과 **파스타 소스 없을때**를 고려하여, **파스타 소스 보관 방법**, **파스타 소스 종류**, 그리고 **파스타 소스 대체품**에 대해 알아보겠습니다. 파스타를 더 맛있게 즐기고자 하는 분들을 위해 유용한 정보를 제공해드리니, 함께 살펴보시기 바랍니다.


### 3.3.3. 제목

In [13]:
# 제목 생성
title = posting_ai.create_title(posting_ai.results['topics'], posting_ai.results['keywords'], save=True)
print(f"🟢 title : {title}")

🟢 title : "파스타 소스 소비기한 및 대체품: 보관 방법과 종류도 알아보세요!"


중간점검

In [14]:
posting_ai.results

{'title': '"파스타 소스 소비기한 및 대체품: 보관 방법과 종류도 알아보세요!"',
 'prologue': '파스타 소스는 다양한 요리에 활용되는 중요한 재료 중 하나입니다. 이 블로그 포스트에서는 **파스타 소스 소비기한**과 **파스타 소스 없을때**를 고려하여, **파스타 소스 보관 방법**, **파스타 소스 종류**, 그리고 **파스타 소스 대체품**에 대해 알아보겠습니다. 파스타를 더 맛있게 즐기고자 하는 분들을 위해 유용한 정보를 제공해드리니, 함께 살펴보시기 바랍니다.',
 'keywords': ['파스타 소스 소비기한', '파스타 소스 소비기한', '파스타 소스 없을때', '파스타 소스 없을때'],
 'theme': '파스타',
 'topics': ['파스타 소스 보관 방법', '파스타 소스 종류', '파스타 소스 대체품'],
 'contents': [],
 'images': [],
 'documents_urls': [],
 'documents': [],
 'html_for_upload': None}

### 3.3.4. 자료수집

3.3.4.1 자료수집

In [15]:
# 자료수집
for topic in posting_ai.results['topics'] :
    print(f"🟢 processing topic : {topic}")
    hrefs, documents = crawler.ddgsearch_reducing(topic)    
    print(f"🟢 {topic}에 관한 문서 {len(documents)}개 수집완료")
    posting_ai.results['documents'].append(documents)
    posting_ai.results['documents_urls'].append(hrefs)    


🟢 processing topic : 파스타 소스 보관 방법
🟢 파스타 소스 보관 방법에 관한 문서 5개 수집완료
🟢 processing topic : 파스타 소스 종류
🟢 파스타 소스 종류에 관한 문서 5개 수집완료
🟢 processing topic : 파스타 소스 대체품
🟢 파스타 소스 대체품에 관한 문서 5개 수집완료


중간점검

3.3.4.2 자료 벡토스토어 저장

In [16]:
# 긁어들인 글들을 벡터스토어에 저장하기
collected_documents = []
for document in posting_ai.results['documents'] :
    collected_documents.extend(document)
posting_ai.vectorstore_recursive_save_texts(collected_documents)

# 결과 출력
print("🟢", len(collected_documents),"개의 문서를 벡터스토어에 저장하였습다..")
print(f"총 {len(posting_ai.vectorstore_list)}개의 문서로 쪼개어 저장하였습니다..")
posting_ai.vectorstore_list.head()


✅ 벡토어스토어 내 새 문서 추가 완료
🟢 15 개의 문서를 벡터스토어에 저장하였습다..
총 327개의 문서로 쪼개어 저장하였습니다..


,data,id
0,파스타소스 소비기한 그냥 버리지 마세요! : 네이버 블로그\n본문 바로가기\n블로그...,fb54322d-21a8-4064-ad16-ab3f4df80766
1,가\n 공감하기\n공유하기 \nURL복사 \n신고하기 \n​제가 너무너무 좋아하는 ...,6edab55f-e4a4-4b5b-8fdc-a93e72078c68
2,"​파스타 면과 소스만 한번 사두면, 면을 삶고 소스를 섞어서 원할 때마다 간단한 방...",72be3882-60ed-45c3-9b35-fa7a44f572e4
3,​​그냥 버리긴 아까워서 파스타소스 소비기한에대해 알아봤는데요. 야호! 다행히도 파...,b7ed0505-579e-47b7-b206-25f3de4e892e
4,​혹시 유통기한과 소비기한의 차이를모르시는 분들 계실까요? 모르시는분들을 위해 간단...,bee007be-ba79-469b-aaa3-56b350cf4e01


결과표시

In [17]:
# 참고문서 호출
q = posting_ai.results['topics'][0]
print(f'"{q}"에 대한 참고문서 검색 결과 :')
posting_ai.vectorstore_similarity_search(q, score_threshold = 0.25, k=10)

"파스타 소스 보관 방법"에 대한 참고문서 검색 결과 :


['파스타 소스의 종류와 선택 방법',
 '파스타소스 냉동 보관방법 남은 소스 페스토 소분보관 : 네이버 블로그\nNAVER\n블로그\n주며느리의 일상\n블로그 검색\n이 블로그에서 검색\n공감\n\t\n\t\n\t댓글\n\t\n\t\n\t\n\t1\n\t\n\t\n공유하기\n메뉴 바로가기\n본문 바로가기\n내 블로그 \n이웃블로그\n블로그 홈 \n로그인\n주며느리의 일상\n블로그 메뉴\n프롤로그\n블로그\n홈카페/인테리어\nIT/전자\n해외여행\n임신/육아\n안부\n\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\n검색\n글 검색\n블로그\n글쓰기\n가벼운 글쓰기툴 퀵에디터가 오픈했어요!\n전체보기 2,063개의 글\r\n\t\n전체보기목록열기\n간단요리',
 '\u200b파스타 소스 보관 방법은 미개봉 기준, 직사광선을 피한 서늘한 곳에서 보관해주시면 돼요. 딱히 냉장 보관이나 냉동보관을 해줄 필요도 없고, 유통기한도길기 때문에 보관이 매우 쉬운 편입니다.\u200b그러나 개봉한 파스타 소스는 냉장 보관을하더라도 보관 기간이 매우 짧은 편! 유통기한이 많이 남았더라도, 개봉을 한 이상보관 기간이 매우 짧아지기 때문에 가급적한 번에 모두 소비하는 것이 좋고요. 한 번에소비를 못했다면 최대한 빠른 시일 내로남은 파스타 소스를 소비해 주셔야 해요.\u200b\u200b\u200b\u200b\u200b',
 '안녕하세요!주며느리 입니다. : )\u200b파스타소스나 아보카도 살사소스모든 소스류를 냉장고에오래 보관하게 되면 뚜껑 주변으로 곰팡이가생기더라구요요 ㅜㅜ 버린 경우가 많아서 너무나 아깝...\u200b그래서!파스타 소스 한번 개봉하면무조건 소분해서 냉동실에 보관하고 있답니다.\u200b파스타 소스 뿐만 아니라 사용하고 남은 모든 소스! 다소분해서 냉동보관하면 보관기간도 길고좋아요~\u200b',
 '전체보기목록열기\n간단요리\n파스타소스 냉동 보관방법 남은 소스 페스토 소분보관 \n주며느리\n ・ \n2023. 9. 

### 3.3.5. 본문

In [18]:
# 글 작성
contents = posting_ai.create_content(
    topics = topics, 
    language='English', 
    score_threshold=0.25,  
    k=10,  
    save=True)

결과표시

In [19]:
posting_ai.results

{'title': '"파스타 소스 소비기한 및 대체품: 보관 방법과 종류도 알아보세요!"',
 'prologue': '파스타 소스는 다양한 요리에 활용되는 중요한 재료 중 하나입니다. 이 블로그 포스트에서는 **파스타 소스 소비기한**과 **파스타 소스 없을때**를 고려하여, **파스타 소스 보관 방법**, **파스타 소스 종류**, 그리고 **파스타 소스 대체품**에 대해 알아보겠습니다. 파스타를 더 맛있게 즐기고자 하는 분들을 위해 유용한 정보를 제공해드리니, 함께 살펴보시기 바랍니다.',
 'keywords': ['파스타 소스 소비기한', '파스타 소스 소비기한', '파스타 소스 없을때', '파스타 소스 없을때'],
 'theme': '파스타',
 'topics': ['파스타 소스 보관 방법', '파스타 소스 종류', '파스타 소스 대체품'],
 'contents': ['### 파스타 소스 보관 소개\n\n파스타 소스는 많은 요리에 다재다능하고 맛있게 추가할 수 있지만 신선도와 풍미를 유지하려면 적절한 보관이 필수적입니다. 집에서 만든 파스타 소스를 다루든 상점에서 구입한 파스타 소스를 다루든 올바른 보관 방법을 알면 수명과 맛에 상당한 차이를 만들 수 있습니다. 파스타 소스를 최대한 오랫동안 신선하고 맛있게 유지하기 위해 올바르게 보관하는 방법에 대해 자세히 알아보겠습니다.\n\n#### 개봉하지 않은 파스타 소스 보관\n\n개봉하지 않은 파스타 소스의 경우 보관 요구 사항은 비교적 간단합니다. 개봉하지 않은 파스타 소스는 직사광선을 피해 서늘하고 어두운 곳에 보관하는 것이 가장 좋습니다. 소스가 안정적인 온도의 식료품 저장실이나 찬장에 보관되는 한 냉장이나 냉동이 필요하지 않습니다.\n\n#### 개봉 파스타 소스 취급\n\n파스타 소스가 담긴 병이나 용기를 열면 보관 요구 사항이 크게 변경됩니다. 개봉한 파스타 소스는 박테리아의 성장을 늦추고 품질을 유지하기 위해 즉시 냉장 보관해야 합니다. 냉장고에 넣기 전에 용기를 뚜껑으로 단단히 밀봉하거나 밀폐

### 3.3.6. 이미지수집

In [20]:
# 이미지 수집하기
crawler = Crawler(verbose=False)
try :
    topics = posting_ai.results['topics']
    for topic in topics : 
        images = crawler.ddgsearch_get_images(topic)
        posting_ai.results['images'].append(images)    
finally :    
    crawler.driver.quit()       



결과표시

In [21]:
posting_ai.results['images']

[[{'title': 'Ultimate Guide to ROME SOUVENIRS & the best gifts from Rome!',
   'image': 'https://www.roamthegnome.com/wp-content/uploads/2020/01/rome-souvenirs-pasta-shop-800.jpg',
   'thumbnail': 'https://tse4.mm.bing.net/th?id=OIP.lzbeSRmykTpp-w-giJ9h1AHaFi&pid=Api',
   'url': 'https://roamthegnome.com/what-to-buy-in-rome-souvenirs/',
   'height': 598,
   'width': 800,
   'source': 'Bing'}],
 [{'title': 'Jars Of Homemade Spaghetti Sauce Free Stock Photo - Public Domain Pictures',
   'image': 'https://www.publicdomainpictures.net/pictures/230000/velka/jars-of-homemade-spaghetti-sauce.jpg',
   'thumbnail': 'https://tse2.mm.bing.net/th?id=OIP.7EGWXtKTEHYm2Br4DtQw8AHaE8&pid=Api',
   'url': 'https://www.publicdomainpictures.net/en/view-image.php?image=222283&picture=jars-of-homemade-spaghetti-sauce',
   'height': 1280,
   'width': 1920,
   'source': 'Bing'}],
 [{'title': 'Free Images : food, blackberry, ingredient, fruit, plant, cuisine ...',
   'image': 'https://c.pxhere.com/images/f6/80

## 3.4 포맷팅하기 (HTML 형식 만들기)

In [22]:
# 글 포맷팅하기
formatted_contents = posting_ai.create_HTML_formmater(save=True)
print(formatted_contents)


        파스타 소스는 다양한 요리에 활용되는 중요한 재료 중 하나입니다. 이 블로그 포스트에서는 **파스타 소스 소비기한**과 **파스타 소스 없을때**를 고려하여, **파스타 소스 보관 방법**, **파스타 소스 종류**, 그리고 **파스타 소스 대체품**에 대해 알아보겠습니다. 파스타를 더 맛있게 즐기고자 하는 분들을 위해 유용한 정보를 제공해드리니, 함께 살펴보시기 바랍니다.
        
                <h2>파스타 소스 보관 방법</h2>                        
            
                    <figure style='align-items: center; text-align : center;'>
                         <img src="https://www.roamthegnome.com/wp-content/uploads/2020/01/rome-souvenirs-pasta-shop-800.jpg" alt="파스타 소스 보관 방법" style='width=400px'>
                        <figcaption>
                            파스타 소스 보관 방법 (출처 : https://roamthegnome.com/what-to-buy-in-rome-souvenirs/)
                        </figcaption>
                    </figure>
                    
                <h3>파스타 소스 보관 소개</h3>

<p>파스타 소스는 많은 요리에 다재다능하고 맛있게 추가할 수 있지만 신선도와 풍미를 유지하려면 적절한 보관이 필수적입니다. 집에서 만든 파스타 소스를 다루든 상점에서 구입한 파스타 소스를 다루든 올바른 보관 방법을 알면 수명과 맛에 상당한 차이를 만들 수 있습니다. 파스타 소스를 최대한 오랫동안 신선하고 맛있게 유지하기 위

# 3.5 저장하기

In [23]:
# 수집 및 생성한 모든 자료를 json으로 저장합니다.
file_manager.saveGeneratedDict2Json(
    data = posting_ai.results,
    subject = theme,
    language = 'ko',
    contents = posting_ai.results['title'],
    folder_category='storage'
)

# 포스팅할 문서를 txt로 저장합니다.
file_manager.saveGeneratedText2Text(
    data = posting_ai.results['html_for_upload'],
    subject = theme,
    language = 'ko',
    contents = posting_ai.results['title'],
    folder_category='for_upload'
)


[Errno 22] Invalid argument: 'c:\\hellodear\\tistory_ebook\\pasta_blog\\창고\\[[파스타]]_((ko))_``"파스타 소스 소비기한 및 대체품: 보관 방법과 종류도 알아보세요!"``.json'
An error occurred: [Errno 22] Invalid argument: 'c:\\hellodear\\tistory_ebook\\pasta_blog\\업로드할것\\[[파스타]]_((ko))_``"파스타 소스 소비기한 및 대체품: 보관 방법과 종류도 알아보세요!"``.txt'
